In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

d:\D-Program Files\Python\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pc\.cache\huggingface\hub\models--google-bert--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer_bert", eval_strategy="epoch")

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

d:\D-Program Files\Python\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.137819,0.496000
2,No log,1.045357,0.559000
3,No log,1.053725,0.587000


TrainOutput(global_step=375, training_loss=1.0365265299479167, metrics={'train_runtime': 92.6222, 'train_samples_per_second': 32.39, 'train_steps_per_second': 4.049, 'total_flos': 789354427392000.0, 'train_loss': 1.0365265299479167, 'epoch': 3.0})

In [11]:
trainer.evaluate()

{'eval_loss': 1.0537248849868774,
 'eval_accuracy': 0.587,
 'eval_runtime': 7.4994,
 'eval_samples_per_second': 133.344,
 'eval_steps_per_second': 16.668,
 'epoch': 3.0}

In [28]:
# print( small_train_dataset[1]['text'] )
small_train_dataset[5]

{'label': 2,
 'text': "Review for the Lounge/Club:\\nEvery time I go to Vegas this is always one of my stops- primarily because of the view. I started going years ago before it was super hyped up and packed almost all the time. I also remember the days when the Witch Doctors were twice the size they are now (and weaker than they used to be!). Voodoo hasn't changed much in the past 5 years except going cheap on the drinks and over packing the club with VIP sections that are unused (which is totally annoying if you're trying to see the view without a bunch of drunk asses running into you). \\n\\nThe DJs are consistently good and it has a very, very mixed crowd. However, I only keep coming back for the view- it doesn't compare to anything else. Ghost Bar has nothing on this place. \\n\\nAlso, over the years the award winning flair bartending has leveled off (I think I saw one bottle throw the whole time we were there) and it's not nearly the show it used to be which is really disappointin

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the fine-tuned model and tokenizer
model_name = 'test_trainer\checkpoint-375'
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Function to generate answers
def generate_answer(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    return predicted_class_id

# Example usage
text = "Review for the Lounge/Club:\\nEvery time I go to Vegas this is always one of my stops- primarily because of the view. I started going years ago before it was super hyped up and packed almost all the time. I also remember the days when the Witch Doctors were twice the size they are now (and weaker than they used to be!). Voodoo hasn't changed much in the past 5 years except going cheap on the drinks and over packing the club with VIP sections that are unused (which is totally annoying if you're trying to see the view without a bunch of drunk asses running into you). \\n\\nThe DJs are consistently good and it has a very, very mixed crowd. However, I only keep coming back for the view- it doesn't compare to anything else. Ghost Bar has nothing on this place. \\n\\nAlso, over the years the award winning flair bartending has leveled off (I think I saw one bottle throw the whole time we were there) and it's not nearly the show it used to be which is really disappointing. If you ever have a chance to catch the episode of Three Sheets on Spike where Zane goes to Vegas- you'll see what I mean. The show was filmed several years ago and even there you can see the Witch Doctor and the flair are way better than they are now. \\n\\nFor newcomers I think this is a great place to at least see once just because the view is so amazing. However, for those who have been going back over and over again- it's turning into a huge disappointment. \\n\\nFor the Restaurant:\\n\\nI've dined here several times and price-wise it runs about the same as any other high end steak house you'd find on the strip. Mostly we ate here out of convenience since the selection at Rio isn't the greatest for food. The service has been reasonably decent and the food was good however, if dinner is going to cost around $100 a person I want to be wowed. The whole experience didn't seem like a 5 star dining like when we went to Strip House (wow! by the way). The servers were friendly but only slightly knowledgeable about the wine list and it seemed like there wasn't enough coverage for the tables which made dinner take forever. I don't really remember much more than that which is disappointing because I wish it were more memorable for the price paid. Been there, done that- won't be back to Voodoo Steak when there's so many better options for the same price."
answer = generate_answer(text)
print(f"Predicted class: {answer}")

<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\c'
C:\Users\pc\AppData\Local\Temp\ipykernel_296\36330538.py:5: SyntaxWarning: invalid escape sequence '\c'
  model_name = 'test_trainer\checkpoint-375'


Predicted class: 1


In [29]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})